In [43]:
# !pip install geopy

In [61]:
import requests 
from geopy.geocoders import Nominatim
from geopy.distance import geodesic

In [62]:
# Get exact latitude and longitude of the address

input_address = '285 Peyton Ct'
geolocator = Nominatim(user_agent="ishita")
location = geolocator.geocode(input_address)

address = location.address
latitude = location.latitude
longitude = location.longitude


print("Address:", address)
print("Latitude:", latitude)
print("Longitude:", longitude)

Address: 285, Peyton Court, Copeley Hill Apartments, University of Virginia, Albemarle County, Virginia, 22903, United States
Latitude: 38.0480118
Longitude: -78.5091330774942


In [63]:
# Using the Overpass API to retrieve map data

overpass_url = "http://overpass-api.de/api/interpreter"

In [74]:
# Function to get grocery stores around a given latitude and longitude

def find_grocery_stores(latitude, longitude, radius=1.0):

    overpass_query = f"""
    [out:json];
    (
      node["shop"="supermarket"](around:{radius * 1000}, {latitude}, {longitude});
      node["shop"="grocery"](around:{radius * 1000}, {latitude}, {longitude});
    );
    out body;
    """

    response = requests.get(overpass_url, params={'data': overpass_query})
    
    if response.status_code == 200:
        data = response.json()
        stores = data['elements']
        
        if not stores:
            print("No grocery stores found within the specified radius.")
        else:
            # Collect and sort grocery stores by distance
            stores_with_distance = []
            for store in stores:
                store_location = (store['lat'], store['lon'])
                store_name = store.get('tags', {}).get('name', 'Unnamed')
                distance = geodesic((latitude, longitude), store_location).km
                stores_with_distance.append((store_name, store_location, distance))
            
            # Sort by distance
            stores_with_distance.sort(key=lambda x: x[2])

            # Print sorted results
            print(f"Found {len(stores)} grocery stores within {radius} km, sorted by distance:")
            for store_name, store_location, distance in stores_with_distance:
                print(f"Name: {store_name}, Coordinates: {store_location}, Distance: {distance:.2f} km")
    else:
        print("Error fetching data from Overpass API. Please try again later.")

In [70]:
# Function to find bus stops around a given latitude and longitude

def find_bus_stops(latitude, longitude, radius=1.0):
    overpass_query = f"""
    [out:json];
    node["highway"="bus_stop"](around:{radius * 1000}, {latitude}, {longitude});
    out body;
    """
    response = requests.get(overpass_url, params={'data': overpass_query})
    
    if response.status_code == 200:
        data = response.json()
        bus_stops = data['elements']
        
        if not bus_stops:
            print("No bus stops found within the specified radius.")
        else:
           
            bus_stops_with_distance = []  # to collect and sort by distance
            for stop in bus_stops:
                stop_location = (stop['lat'], stop['lon'])
                stop_name = stop.get('tags', {}).get('name', 'Unnamed')
                distance = geodesic((latitude, longitude), stop_location).km
                bus_stops_with_distance.append((stop_name, stop_location, distance))
        

            bus_stops_with_distance.sort(key=lambda x: x[2]) # sorting by distance

            print(f"Found {len(bus_stops)} bus stops within {radius} km, sorted by distance:")
            for stop_name, stop_location, distance in bus_stops_with_distance:
                print(f"Name: {stop_name}, Coordinates: {stop_location}, Distance: {distance:.2f} km")
    else:
        print("Error fetching data from Overpass API. Please try again later.")

In [ ]:
find_grocery_stores(latitude, longitude, radius=2.0)  # within a radius of 2 km

Found 5 grocery stores within 2.0 km, sorted by distance:
Name: Harris Teeter, Coordinates: (38.0499778, -78.5042131), Distance: 0.48 km
Name: Foods of All Nations, Coordinates: (38.0431314, -78.5119486), Distance: 0.60 km
Name: Kroger, Coordinates: (38.0535319, -78.5006204), Distance: 0.97 km
Name: Asian Market, Coordinates: (38.0561993, -78.4966832), Distance: 1.42 km
Name: Super Amanecer, Coordinates: (38.0562741, -78.4966307), Distance: 1.43 km


In [73]:
find_bus_stops(latitude, longitude, radius=0.5)  # within a radius of 0.5 km

Found 8 bus stops within 0.5 km, sorted by distance:
Name: Massie Rd @ JPJ West Lot, Coordinates: (38.0462614, -78.5093282), Distance: 0.20 km
Name: Massie Rd @ JPJ South Lot, Coordinates: (38.0458151, -78.5086629), Distance: 0.25 km
Name: Massie Rd @ Faulkner Apts (Northbound), Coordinates: (38.048134, -78.51205), Distance: 0.26 km
Name: Arlington Blvd @ Millmont St, Coordinates: (38.049372, -78.506336), Distance: 0.29 km
Name: Massie Rd @ John Paul Jones Arena, Coordinates: (38.0450453, -78.5066476), Distance: 0.39 km
Name: Massie Rd @ North Grounds Rec (Northbound), Coordinates: (38.0506433, -78.5124984), Distance: 0.42 km
Name: Massie Rd @ Copeley Apts, Coordinates: (38.051897, -78.509797), Distance: 0.44 km
Name: Massie Rd @ JPJ East Lot, Coordinates: (38.0447249, -78.5061906), Distance: 0.45 km
